In [8]:
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import torch

import sys
sys.path.append('/home/nauel/bert_gender_bias')
import warnings
warnings.filterwarnings("ignore")


from pipelines.utils.paths import EXTERNAL_DATA_DIR, INTERIM_DATA_DIR

In [18]:
gendered_words = pd.read_pickle(os.path.join(INTERIM_DATA_DIR, 'gender_binary_words_TOKEN.pkl'))
jobs_df = pd.read_pickle(os.path.join(INTERIM_DATA_DIR, 'occupations_TOKEN.pkl'))

In [117]:
gendered_words.dropna(inplace=True)
print(gendered_words.shape)
gendered_words.head()

(431, 6)


,word,gender_binary,bert_token,word2vec_token,word_count,shape
0,abbot,0,"[-0.39571184, -0.093838364, 0.06868138, 0.1723...","[0.40039062, 0.41015625, 0.36523438, 0.2207031...",1,300
1,abbots,0,"[-0.18790531, -0.077521764, -0.3876859, 0.1816...","[0.30664062, 0.33398438, 0.1953125, 0.50390625...",1,300
2,adulterer,0,"[-0.45352724, -0.38397712, -0.25277817, -0.266...","[0.42773438, -0.28515625, -0.0625, 0.020263672...",1,300
3,adulterers,0,"[-0.17249976, -0.15101261, -0.3717648, -0.0279...","[-0.14160156, -0.22558594, -0.042236328, 0.208...",1,300
4,airman,0,"[-0.20044291, -0.020562049, -0.276353, -0.0372...","[0.49023438, -0.15917969, -0.022460938, -0.024...",1,300


In [118]:
print(jobs_df.shape)
jobs_df.head()

(378, 4)


,Occupations,bert_token,word2vec_token,word_count
0,Accountant,"[0.044247508, -0.062065974, -0.28353956, 0.019...","[0.059326172, -0.1328125, -0.16796875, -0.1816...",1
6,Actor,"[0.104604356, -0.23240899, -0.5950049, -0.1360...","[0.33789062, -0.028442383, 0.111328125, -0.181...",1
7,Actress,"[0.044713546, -0.2707646, -0.26443234, -0.0969...","[0.24023438, -0.20214844, 0.16894531, -0.20410...",1
8,Actuary,"[-0.102212645, -0.09090995, -0.14266734, -0.08...","[0.032714844, -0.41210938, -0.14550781, 0.3691...",1
9,Acupuncturist,"[0.05669831, -0.22730672, -0.20539528, -0.2271...","[0.16894531, -0.29296875, 0.007507324, -0.1503...",1


In [119]:
token_cols = ['bert_token', 'word2vec_token']

import sklearn

from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

for token_col in token_cols:
    print(token_col)
    
    X = gendered_words[token_col].values.tolist()
    y = np.array(gendered_words.gender_binary.tolist())
    clf = make_pipeline(StandardScaler(),
                        LinearSVC(random_state=0, tol=1e-5))
    clf.fit(X, y)

    print("Accuracy: ", clf.score(X, y))
    

bert_token
Accuracy:  0.9976798143851509
word2vec_token
Accuracy:  0.9976798143851509
